In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib
import os
import json
from typing import Tuple

In [2]:
"""
[{
  "header": "YouTube",
  "title": "サイバネティクス・リアリティ工学研究室からのメッセージ を視聴しました",
  "titleUrl": "https://www.youtube.com/watch?v\u003dvF1lQMUhjyE",
  "subtitles": [{
    "name": "奈良先端大 公式チャンネル",
    "url": "https://www.youtube.com/channel/UCSWlIFHU1GSo7iqRVEH-LHA"
  }],
  "time": "2024-11-06T13:23:25.209Z",
  "products": ["YouTube"],
  "activityControls": ["YouTube の再生履歴"]
},{
  "header": "YouTube",
  "title": "reel2023 を視聴しました",
  "titleUrl": "https://www.youtube.com/watch?v\u003d6TyHOUAVPH0",
  "subtitles": [{
    "name": "KajimotoLab",
    "url": "https://www.youtube.com/channel/UCAQrGH60EW1Lkl_gm-73e_A"
  }],
  "time": "2024-11-06T13:17:19.861Z",
  "products": ["YouTube"],
  "activityControls": ["YouTube の再生履歴"]
}
]
"""

def subtitles_to_str(subtitles) -> Tuple[str, str]:
    if not isinstance(subtitles, list) or len(subtitles) == 0:
        return ("", "")
    else:
        return (subtitles[0].get("name", ""), subtitles[0].get("url", ""))

csv_file_name = "data.csv"

files = os.listdir("data")
dfs = []
for file in files:
    with open(f"data/{file}", "r") as f:
        json_data = json.load(f)
    df = pd.DataFrame(json_data)
    for i in range(len(df)):
        df.loc[i, "subtitles_name"], df.loc[i, "subtitles_url"] = subtitles_to_str(df.loc[i, "subtitles"])
    dfs.append(df)

df = pd.concat(dfs)
df.to_csv(csv_file_name, index=False)

In [3]:
# データの読み込み
df = pd.read_csv(csv_file_name)

In [4]:
# データの数を確認
print(df.shape)

(60446, 11)


In [ ]:
# 年毎の数をプロット
df["time"] = pd.to_datetime(df["time"], format='ISO8601')
df["year"] = df["time"].dt.year
df["month"] = df["time"].dt.month

df_year = df.groupby("year").size()
df_month = df.groupby(["year", "month"]).size().unstack(fill_value=0)

fig = plt.figure(figsize=(8, 4))
plt.title("年毎のアクティビティ視聴動画数")
# plt.plot(df_year.index, df_year.values, marker='o')
plt.bar(df_year.index, df_year.values)
plt.xlabel("年")
plt.ylabel("視聴動画数")
plt.xticks(df_year.index)
# plt.grid(True)
# plt.tight_layout()
plt.savefig("activity.png")
plt.show()

fig = plt.figure(figsize=(8, 6))
plt.title("月毎のアクティビティ視聴動画数")
df_month.T.plot(kind="line", ax=plt.gca(), marker='o')
# df_month.plot(kind="bar", stacked=True, ax=plt.gca())
plt.xlabel("月")
plt.ylabel("視聴動画数")
# plt.xticks(range(1, 13))
plt.grid(True)
plt.tight_layout()
plt.savefig("activity_month.png")
plt.show()

fig = plt.figure(figsize=(12, 6))
plt.title("時系列での月ごとのアクティビティ視聴動画数")
df_month.plot(kind="line", ax=plt.gca(), marker='o')
plt.xlabel("年")
plt.ylabel("視聴動画数")
plt.xticks(df_month.index)
plt.grid(True)
plt.tight_layout()
plt.savefig("activity_month_time.png")
plt.show()

# 最初の年の最初の月から順番に数を折れ線グラフとしてプロット
df_year_month = df.groupby(["year", "month"]).size()
df_year_month = df_year_month.reset_index()
df_year_month["date"] = pd.to_datetime(df_year_month[["year", "month"]].assign(day=1))
df_year_month = df_year_month.sort_values("date")

fig = plt.figure(figsize=(16, 6))
plt.title("年代別youtube視聴動画数")
plt.plot(df_year_month["date"], df_year_month[0], marker='o')
# plt.bar(df_year_month["date"], df_year_month[0])
plt.xlabel("年月")
plt.ylabel("視聴動画数")
plt.xticks(df_year_month["date"], rotation=45)
plt.grid(True)
plt.tight_layout()
plt.savefig("activity_month_time2.png")
plt.show()

fig = plt.figure(figsize=(16, 6))
plt.title("年代別youtube視聴動画数")
plt.bar(df_year_month["date"], df_year_month[0], width=20)
plt.xlabel("年月")
plt.ylabel("視聴動画数")
plt.xticks(df_year_month["date"], rotation=45)
plt.grid(True, axis='y')
plt.tight_layout()
plt.savefig("activity_month_time2.png")
plt.show()

In [ ]:
# 年毎の数をプロット
df["time"] = pd.to_datetime(df["time"], format='ISO8601')
df["year"] = df["time"].dt.year
df["month"] = df["time"].dt.month

df_year = df.groupby("year").size().cumsum()
df_month = df.groupby(["year", "month"]).size().unstack(fill_value=0).cumsum(axis=1)

fig = plt.figure(figsize=(12, 6))
plt.title("年毎の累積アクティビティ視聴動画数")
plt.plot(df_year.index, df_year.values, marker='o')
plt.xlabel("年")
plt.ylabel("累積視聴動画数")
plt.xticks(df_year.index)
plt.grid(True)
plt.tight_layout()
plt.savefig("cumulative_activity.png")
plt.show()

# 最初の年の最初の月から順番に数を累積折れ線グラフとしてプロット
df_year_month = df.groupby(["year", "month"]).size().cumsum()
df_year_month = df_year_month.reset_index()
df_year_month["date"] = pd.to_datetime(df_year_month[["year", "month"]].assign(day=1))
df_year_month = df_year_month.sort_values("date")

fig = plt.figure(figsize=(12, 6))
plt.title("これまでの累積youtube視聴動画数", fontsize=16)
plt.plot(df_year_month["date"], df_year_month[0], marker='o')
plt.xlabel("年月", fontsize=16)
plt.ylabel("累積視聴動画数", fontsize=16)
plt.xticks(df_year_month["date"], rotation=45)
plt.grid(True)
plt.gca()
plt.tight_layout()
plt.savefig("cumulative_activity_month_time.png")
plt.show()

In [ ]:
# タイトルが全く同じものをカウント
df_title = df.groupby("title").size().sort_values(ascending=False)

limit = 32

# タイトルから"を視聴しました"を削除
titles = [title.replace("を視聴しました", "") for title in df_title.index[:limit]]

fig = plt.figure(figsize=(12, 8))
plt.title("タイトルごとの視聴動画数")
plt.barh(titles[::-1], df_title.values[:limit][::-1])
plt.xlabel("視聴動画数")
# plt.ylabel("タイトル")
plt.yticks(rotation=0)
plt.grid(True, axis='x')
plt.tight_layout()
plt.savefig("activity_title_horizontal.png")
plt.show()

In [ ]:
# subtitles_name ごとの視聴動画数
df_title = df.groupby("subtitles_name").size().sort_values(ascending=False)

limit = 30

# タイトルから"を視聴しました"を削除
titles =  df_title.index[3:limit]

fig = plt.figure(figsize=(12, 10))
plt.title("チャンネルごとの視聴動画数")
plt.barh(titles[::-1], df_title.values[3:limit][::-1])
plt.xlabel("視聴動画数")
# plt.ylabel("タイトル")
plt.yticks(rotation=0)
plt.grid(True, axis='x')
plt.tight_layout()
plt.savefig("activity_title_horizontal.png")
plt.show()

In [ ]:
# 2024年に視聴した動画のみを抽出
df_2024 = df[df["year"] == 2024]

# タイトルが全く同じものをカウント  
df_title = df_2024.groupby("title").size().sort_values(ascending=False)

limit = 32

# タイトルから"を視聴しました"を削除
titles = [title.replace("を視聴しました", "") for title in df_title.index[:limit]]

fig = plt.figure(figsize=(12, 10))
plt.title("2024年のタイトルごとの視聴動画数")
plt.barh(titles[::-1], df_title.values[:limit][::-1])
plt.xlabel("視聴動画数")
# plt.ylabel("タイトル")
plt.yticks(rotation=0)
plt.grid(True, axis='x')
plt.tight_layout()
plt.savefig("activity_title_2024_horizontal.png")
plt.show()

# subtitles_name ごとの視聴動画数
df_title = df_2024.groupby("subtitles_name").size().sort_values(ascending=False)

limit = 30

# タイトルから"を視聴しました"を削除
titles =  df_title.index[3:limit]

fig = plt.figure(figsize=(12, 10))
plt.title("2024年のチャンネルごとの視聴動画数")
plt.barh(titles[::-1], df_title.values[3:limit][::-1])
plt.xlabel("視聴動画数")
# plt.ylabel("タイトル")
plt.yticks(rotation=0)
plt.grid(True, axis='x')
plt.tight_layout()
plt.savefig("activity_title_2024_horizontal.png")
plt.show()

In [ ]:
# 2023年に視聴した動画のみを抽出
df_2023 = df[df["year"] == 2023]

# タイトルが全く同じものをカウント  
df_title = df_2023.groupby("title").size().sort_values(ascending=False)

limit = 32

# タイトルから"を視聴しました"を削除
titles = [title.replace("を視聴しました", "") for title in df_title.index[:limit]]

fig = plt.figure(figsize=(12, 10))
plt.title("2023年のタイトルごとの視聴動画数")
plt.barh(titles[::-1], df_title.values[:limit][::-1])
plt.xlabel("視聴動画数")
# plt.ylabel("タイトル")
plt.yticks(rotation=0)
plt.grid(True, axis='x')
plt.tight_layout()
plt.savefig("activity_title_2024_horizontal.png")
plt.show()

# subtitles_name ごとの視聴動画数
df_title = df_2023.groupby("subtitles_name").size().sort_values(ascending=False)

limit = 30

# タイトルから"を視聴しました"を削除
titles =  df_title.index[3:limit]

fig = plt.figure(figsize=(12, 10))
plt.title("2023年のチャンネルごとの視聴動画数")
plt.barh(titles[::-1], df_title.values[3:limit][::-1])
plt.xlabel("視聴動画数")
# plt.ylabel("タイトル")
plt.yticks(rotation=0)
plt.grid(True, axis='x')
plt.tight_layout()
plt.savefig("activity_title_2024_horizontal.png")
plt.show()

In [ ]:
# 2023 + 2024
df_2023_2024 = df[(df["year"] == 2023) | (df["year"] == 2024)]

# タイトルが全く同じものをカウント
df_title = df_2023_2024.groupby("title").size().sort_values(ascending=False)

limit = 32

# タイトルから"を視聴しました"を削除
titles = [title.replace("を視聴しました", "") for title in df_title.index[:limit]]

fig = plt.figure(figsize=(12, 10))
plt.title("2023年と2024年のタイトルごとの視聴動画数")
plt.barh(titles[::-1], df_title.values[:limit][::-1])
plt.xlabel("視聴動画数")
# plt.ylabel("タイトル")
plt.yticks(rotation=0)
plt.grid(True, axis='x')
plt.tight_layout()
plt.savefig("activity_title_2023_2024_horizontal.png")
plt.show()

# subtitles_name ごとの視聴動画数
df_title = df_2023_2024.groupby("subtitles_name").size().sort_values(ascending=False)

limit = 30

# タイトルから"を視聴しました"を削除
titles =  df_title.index[3:limit]

fig = plt.figure(figsize=(12, 10))
plt.title("2023年と2024年のチャンネルごとの視聴動画数")
plt.barh(titles[::-1], df_title.values[3:limit][::-1])
plt.xlabel("視聴動画数")
# plt.ylabel("タイトル")
plt.yticks(rotation=0)
plt.grid(True, axis='x')
plt.tight_layout()
plt.savefig("activity_title_2023_2024_horizontal.png")
plt.show()

In [ ]:
# 曜日毎の視聴動画数
df["weekday"] = df["time"].dt.weekday
df_weekday = df.groupby("weekday").size()

fig = plt.figure(figsize=(8, 4))
plt.title("曜日ごとの視聴動画数")
plt.bar(df_weekday.index, df_weekday.values)
plt.xlabel("曜日")
plt.ylabel("視聴動画数")
plt.xticks(df_weekday.index, ["月", "火", "水", "木", "金", "土", "日"])
plt.grid(True)
plt.tight_layout()
plt.savefig("activity_weekday.png")
